In [3]:
from scripts.utility_scripts import create_closure, create_closure_func

import numpy as np
from numpy.lib.stride_tricks import sliding_window_view

# from custom_pytorch_datasets import CorticomExperimentDataset

def _generate_trial_sxx_data(data_func, indices):
    sxx_data_dict = data_func()
    
    sxx = sxx_data_dict['sxx']
    
    return sxx[:, indices, :].astype(np.float16)
    
def _window_spectrogram(sxx, shift, window_size):
    return np.moveaxis(sliding_window_view(sxx, window_shape=window_size, axis=1), [0, -1], [1, -2])[::shift, ...]

def generate_model_sxx_data(spectrogram_dict, curated_states_partition, sessions, model_data_params, current_experiment, patient_id):
    pre_stimulus_time = model_data_params['pre_stimulus_time']
    post_completion_time = model_data_params['post_completion_time']

    window_size = model_data_params['window_size']
    shift = model_data_params['shift']

    model_data_dict = {}
    model_data_filenames_dict = {}
    global_trial_idx = 0
    model_data_filenames = []
    for session_type, session_data in sessions[patient_id][current_experiment].items():
        for sxx_partition_key, sxx_partition_func in spectrogram_dict.items():
            date = sxx_partition_key.split('_')[-2]
            session = sxx_partition_key.split('_')[-1]

            if date in session_data.keys() and session in session_data[date]:
                curated_states_data_func = curated_states_partition[sxx_partition_key]
                curated_states_dict = curated_states_data_func()
                # sxx_data_dict = sxx_partition_func()

                # sampling_rate = sxx_data_dict['sampling_rate']
#                 sampling_rate = 1000

                model_data_metadata = {}
                for state, state_information in curated_states_dict.items():
                    cur_dict = {}

                    start_end_idx = state_information['start_end_idx']
                    
                    sampling_rate = state_information['sampling_rate']
                    
                    samples_pre = int(np.ceil((pre_stimulus_time * sampling_rate)/shift))
                    samples_post = int(np.ceil((post_completion_time * sampling_rate)/shift)) 
                    
                    cur_dict['start_end_idx'] = [(x[0]-samples_pre, x[1]+samples_post) for x in start_end_idx]
                    cur_dict['unique_val_idx'] = [np.arange(x[0], x[1]+1) for x in cur_dict['start_end_idx']]
                    cur_dict['num_steps'] = [len(x) for x in cur_dict['unique_val_idx']]

                    model_data_metadata[state] = cur_dict

                # sxx = sxx_data_dict['sxx']

                intermed_model_dict = {}
                for state, metadata_dict in model_data_metadata.items():
                    if state == 0:
                        continue
                        
                    trial_idx_list = []
                    local_trial_idx_list = []
                    for trial_idx, indices in enumerate(metadata_dict['unique_val_idx']):
                        model_data_dict[f"{session_type}_{date}_{session}_T{trial_idx}_state{state}"] = create_closure_func(_generate_trial_sxx_data, sxx_partition_func, indices)
                        model_data_filenames.append(f"{session_type}_{date}_{session}_T{trial_idx}_state{state}")

                        trial_idx_list.append(global_trial_idx)
                        global_trial_idx += 1
                        
                        local_trial_idx_list.append(trial_idx)
                        
                    cur_partition_dict = model_data_filenames_dict.setdefault(sxx_partition_key, {})

                    cur_partition_dict[state] = {
                            'date': date,
                            'session_type': session_type,
                            'session': session,
                            'state': state,
                            'local_trials_idx_list': local_trial_idx_list,
                            'global_trials_idx_list': trial_idx_list
                    }
                        
    return model_data_dict, model_data_filenames_dict, model_data_filenames

In [4]:
curated_states_partition = catalog.load("center_out_curated_states_pkl")
spectrogram_dict = catalog.load("center_out_spectrogram_std_pkl")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_301137/1448693614.py:1 in <cell line: 1>                                          │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_301137/1448693614.py'                       │
│                                                                                                  │
│ /home/sshah/bci-pipeline/lib/python3.8/site-packages/kedro/io/data_catalog.py:341 in load        │
│                                                                                                  │
│   338 │   │   │   >>> df = io.load("cars")                                                       │
│   339 │   │   """                                                                                │
│   340 │   │   load_version = Version(version, None) if version else None                         │
│ ❱ 341 │   │   dataset = self._get_dataset(name, version=load_version)                            │
│   342 │   │                                                                                      │
│   343 │   │   self._logger.info(                                                                 │
│   344 │   │   │   "Loading data from '%s' (%s)...", name, type(dataset).__name__                 │
│                                                                                                  │
│ /home/sshah/bci-pipeline/lib/python3.8/site-packages/kedro/io/data_catalog.py:300 in             │
│ _get_dataset                                                                                     │
│                                                                                                  │
│   297 │   │   │   │   │   suggestions = ", ".join(matches)                                       │
│   298 │   │   │   │   │   error_msg += f" - did you mean one of these instead: {suggestions}"    │
│   299 │   │   │                                                                                  │
│ ❱ 300 │   │   │   raise DataSetNotFoundError(error_msg)                                          │
│   301 │   │                                                                                      │
│   302 │   │   data_set = self._data_sets[data_set_name]                                          │
│   303 │   │   if version and isinstance(data_set, AbstractVersionedDataSet):                     │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
DataSetNotFoundError: DataSet 'center_out_curated_states_pkl' not found in the catalog - did you mean one of these 
instead: center_out_model_spectrogram_std_pkl, center_out_train_test_indices